In [1]:
import numpy as np
import pandas as pd
#from pandas_datareader import DataReader
import pandas_datareader
from datetime import datetime
import matplotlib.pyplot as plt

import statsmodels.formula.api as sm

import ipywidgets as widgets

%matplotlib inline

In [3]:
LIST = ['JILL','ELTK','ONVO','UAVS','AEY','OBLN','XRF','MLSS','MICT','SOLY','NMRD','ZFGN','INUV','VTGN','RBZ','BURG','MTSL','DXR','QADA','MYOV','BOSC','APTO','PESI','TITN','EYEG','CAPR','UROV','APDN','TTPH','ECOR','WYND','ZN','OSN','HDSN','BDR','TBLT','PLM','DMRC','MOV','PVH','TLYS','CPAH','VLRX','SDRL','RWLK','VEEV','OPRX','RAVE','PFSW','SPWH','TMDX','DOOO','TC','NBY','WSTL','KEYS','DBI','INAP','RVLT','JASN','CNET','RVLV','REKR','SUMR','MBOT','BDGE','FOLD','SRRA','IDN','PIXY','AAC','LYL','MHLD','NIO','SLS','NNVC','MYT','BYND','CREG','ATIS','TCCO','AAMC','OXBR','ANCN','BCRX','ALRN','RTW','LTBR','WWR','CTRC','VTVT','MACK','NDRA','OTLK','MYT','BYND','CREG','ATIS','TCCO','AAMC','OXBR','ANCN','BCRX','ALRN','RTW','LTBR','WWR','HTHT','CTRC','MACK']
List = sorted(list(set(LIST)))
#print(List)

output = widgets.Output()

cal_Start = widgets.DatePicker(
        description='Pick a Start Date',
        disabled=False
    )

cal_End = widgets.DatePicker(
        description='Pick a End Date',
        disabled=False
    )

def dates1(start_date1):
    
    global filtered
    output.clear_output()
    global sd1
    sd1 = start_date1
    
def dates2(end_date1):
    
    global filtered
    output.clear_output()
    
    returns_port = pd.DataFrame()
    for i in List:
        one_stock = pandas_datareader.DataReader(i,'yahoo',sd1,end_date1)
        returns_one_stock = pd.DataFrame(np.diff(np.log(one_stock['Adj Close'].values)))
        #dat1 = pd.concat([dat1, dat2], axis=1)
        returns_port = pd.concat([returns_port, returns_one_stock], axis=1)
    returns_port.columns = List
    
    portfolios = np.mat(returns_port)

    FAMA_FRENCH = pd.read_csv('C:/Users/mohdn/Downloads/F-F_Research_Data_Factors_daily.csv')
    
    sy,sm,sd = sd1.year,'%02d' % sd1.month,'%02d' % sd1.day
    symd = str(sy)+str(sm)+str(sd)
    symd = int(symd)
    
    
    ey,em,ed = end_date1.year,'%02d' % end_date1.month,'%02d' % end_date1.day
    print(ey,em,ed)
    eymd = str(ey)+str(em)+str(ed)
    eymd = int(eymd)
    print(eymd)

    FAMA = FAMA_FRENCH[(FAMA_FRENCH['Date']>symd) & (FAMA_FRENCH['Date']<=eymd)]
    
    print(FAMA)
    
    MKT = pd.DataFrame(FAMA['Mkt-RF'] -  FAMA['RF'])
    SMB = pd.DataFrame(FAMA['SMB'])
    HML = pd.DataFrame(FAMA['HML'])
    
    MKT.reset_index(drop=True,inplace=True)
    SMB.reset_index(drop=True,inplace=True)
    HML.reset_index(drop=True,inplace=True)

    [T,N] = portfolios.shape

    m=np.ones((T,1))

    F = np.concatenate((m,MKT,SMB,HML),axis=1)

    factors = np.mat(F)

    K = F.shape[1]

    beta = np.mat(np.zeros((K,N)))
    
    for j in range(portfolios.shape[1]):
        x = factors
        y = portfolios[:,[j]]
        beta[:,[j]] = np.matmul(np.matmul(np.linalg.inv(np.matmul(x.T,x)),x.T),y) 
    #Step 2. RUN CROSS-SECTIONAL REGRESSION
    # ACTIVE RETURNS
    lamb = np.zeros((K,T))
    alpha = np.zeros((N,T))
    sigma = np.zeros((T,N,N))
    
    for i in range(portfolios.shape[0]):
        x = beta.T
        y = portfolios[[i],:].T
        lambdafun = np.matmul(np.matmul(np.linalg.inv(np.matmul(x.T,x)),x.T),y)
        lamb[:,[i]] = lambdafun
        alpha[:,[i]] = np.subtract(y,np.matmul(x,lambdafun))
        sigma[[i],:,:] = np.matmul(alpha[:,[i]],alpha[:,[i]].T)
    
    ALPHA = alpha.mean(axis =1)
    SIGMA = sigma.mean(axis =0)
    
    R = ALPHA
    R = np.reshape(R,(N,1))
    I = np.ones((N,1))
    V = SIGMA
    V_inv = np.linalg.inv(V)
    
    a = float(np.matmul(np.matmul(R.T,V_inv),R))
    b = float(np.matmul(np.matmul(R.T,V_inv),I))
    c = float(np.matmul(np.matmul(I.T,V_inv),I))
    
    A = np.matrix([[a,b],[b,c]],dtype = 'float')
    A_inv = np.linalg.inv(A)
    
    r_star = (b/c)
    
    lamda_halfs = np.matmul(A_inv,([[r_star],[1]]))
    
    x = float(lamda_halfs[0])*np.matmul(V_inv,R) + float(lamda_halfs[1])*np.matmul(V_inv,I)
    print(sum(x))
    
    New_df = pd.DataFrame(ALPHA,columns=['Values'])
    New_df['Stock'] = List
    plt.rcParams["figure.figsize"] = (27,20)
    plt.subplot(2, 1, 1)
    active_return_plot = plt.stem(New_df['Stock'] , New_df['Values'])
    plt.xticks(
        rotation=90, 
        horizontalalignment='right',
        fontweight='light',
        fontsize='x-large'
    )
    
    active_rate = pd.DataFrame( list(x),columns=['Weights'])
    active_rate['Stocks'] = List
    active_rate['Risk'] = np.sqrt(list(np.diag(SIGMA)))
    top10_active = active_rate.nlargest(10,'Weights')


    #plt.rcParams["figure.figsize"] = (15,15)
    #plt.stem(active_rate['Stocks'] , active_rate['Risk'])
    
    plt.rcParams["figure.figsize"] = (15,15)
    plt.subplot(2, 1, 2)
    
    top10_active_plot = plt.stem(top10_active['Stocks'] , top10_active['Risk'])
    
    
    with output:
        #print(returns_port.head())
        #print(ALPHA)
        #print(SIGMA)
        print(T)
        print(active_return_plot)
        print(top10_active_plot)
        
def dropdown_field_eventhandler(change):
    dates1(change.new)
    
def dropdown_field_eventhandler1(change1):
    dates2(change1.new)
    
cal_Start.observe(dropdown_field_eventhandler, names='value')
cal_End.observe(dropdown_field_eventhandler1, names='value')

display(cal_Start,cal_End)

display(output)    

DatePicker(value=None, description='Pick a Start Date')

DatePicker(value=None, description='Pick a End Date')

Output()

In [ ]:
sda = widgets.DatePicker(
 description='Pick a Start Date',
        disabled=False
    )
eda = widgets.DatePicker(
 description='Pick a Start Date',
        disabled=False
    )
display(sda)
display(eda)

FAMA_FRENCH = pd.read_csv('C:/Users/mohdn/Downloads/F-F_Research_Data_Factors_daily.csv')
sy,sm,sd = sda.value.year,'%02d' % sda.value.month,'%02d' % sda.value.day
print(sy,sm,sd)
symd = str(sy)+str(sm)+str(sd)
symd = int(symd)
print(symd)

ey,em,ed = eda.value.year,'%02d' % eda.value.month,'%02d' % eda.value.day
print(ey,em,ed)
eymd = str(ey)+str(em)+str(ed)
eymd = int(eymd)
print(eymd)

#FAMA_FRENCH.set_index('Date',inplace = True)

#FF = FAMA_FRENCH.iloc[ymd]
FF = FAMA_FRENCH[(FAMA_FRENCH['Date']>symd) & (FAMA_FRENCH['Date']<=eymd)]
len(FF)